#### Link of the Pycaret docs : https://pycaret.readthedocs.io/en/latest/api/time_series.html

In [1]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [2]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbgraos_world.xls', sheet_name='Planilha1')
print(df.head(10))

   date   producao  disponibilidade   tipo
0  2017  198573810      125905263.0   soja
1  2018  195252258      124131656.0   soja
2  2019  193676716      121360171.0   soja
3  2020  199275404      127453417.0   soja
4  2021  205694243      130477261.0   soja
5  2022  203470007      133791633.0   soja
6  2017  125905263              NaN  milho
7  2018  124131656              NaN  milho
8  2019  121360171              NaN  milho
9  2020  127453417              NaN  milho


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             12 non-null     int64  
 1   producao         12 non-null     int64  
 2   disponibilidade  6 non-null      float64
 3   tipo             12 non-null     object 
dtypes: float64(1), int64(2), object(1)
memory usage: 512.0+ bytes


### Avaliando a produção de soja

In [4]:
df.drop(['disponibilidade'], axis = 1,inplace=True)
df = df[df['tipo']=='soja']
df.head(10)


,date,producao,tipo
0,2017,198573810,soja
1,2018,195252258,soja
2,2019,193676716,soja
3,2020,199275404,soja
4,2021,205694243,soja
5,2022,203470007,soja


In [5]:
df.drop(['tipo'], axis = 1,inplace=True)

In [6]:
df.set_index('date',inplace=True)

In [7]:
df=df.sort_index(axis=0)
print(df.head(10))

       producao
date           
2017  198573810
2018  195252258
2019  193676716
2020  199275404
2021  205694243
2022  203470007


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. When integer is passed it means N continuous points in the future without any gap. If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [8]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,producao
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(6, 1)"
5,Transformed data shape,"(6, 1)"
6,Transformed train set shape,"(5, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [9]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Checking Train-Test Split

In [10]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [11]:
top4 = exp.compare_models(n_select = 4)

## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [12]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":10})

###  ETS

In [13]:
ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,0.3024,0.3024,1004336.6577,1004336.6577,0.0052,0.0052
1,2019.0000,2.9646,2.7924,7259023.0561,7259023.0561,0.0364,0.0371
2,2020.0000,2.2744,2.0577,7957064.7892,7957064.7892,0.0387,0.0394
Mean,nan,1.8471,1.7175,5406808.1677,5406808.1677,0.0268,0.0272
SD,nan,1.1281,1.0446,3126033.9209,3126033.9209,0.0153,0.0156


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## exp_smooth

In [14]:
exp_smooth  = exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## grand_means

In [15]:
grand_means  = exp.create_model('grand_means')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = grand_means, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,0.9743,0.9743,3236318.0000,3236318.0000,0.0167,0.0166
1,2019.0000,1.4054,1.3238,3441142.6667,3441142.6667,0.0173,0.0174
2,2020.0000,2.5724,2.3273,8999696.0000,8999696.0000,0.0438,0.0447
Mean,nan,1.6507,1.5418,5225718.8889,5225718.8889,0.0259,0.0262
SD,nan,0.6751,0.5735,2669914.5695,2669914.5695,0.0126,0.0131


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## naive

In [16]:

naive = exp.create_model('naive')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = naive, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,0.4743,0.4743,1575542.0000,1575542.0000,0.0081,0.0081
1,2019.0000,2.2865,2.1537,5598688.0000,5598688.0000,0.0281,0.0285
2,2020.0000,1.8347,1.6599,6418839.0000,6418839.0000,0.0312,0.0317
Mean,nan,1.5319,1.4293,4531023.0000,4531023.0000,0.0225,0.0228
SD,nan,0.7702,0.7047,2116492.8336,2116492.8336,0.0102,0.0105


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


#### croston

In [17]:

croston = exp.create_model('croston')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = croston, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ALL GRAPHS

In [18]:
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = grand_means, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = naive, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = croston, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})


               

## Avaliando a produção do milho 

In [19]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbgraos_world.xls', sheet_name='Planilha1')
print(df.head(10))

   date   producao  disponibilidade   tipo
0  2017  198573810      125905263.0   soja
1  2018  195252258      124131656.0   soja
2  2019  193676716      121360171.0   soja
3  2020  199275404      127453417.0   soja
4  2021  205694243      130477261.0   soja
5  2022  203470007      133791633.0   soja
6  2017  125905263              NaN  milho
7  2018  124131656              NaN  milho
8  2019  121360171              NaN  milho
9  2020  127453417              NaN  milho


In [20]:
df.drop(['disponibilidade'], axis = 1,inplace=True)
df = df[df['tipo']=='milho']
df.head(10)

,date,producao,tipo
6,2017,125905263,milho
7,2018,124131656,milho
8,2019,121360171,milho
9,2020,127453417,milho
10,2021,130477261,milho
11,2022,133791633,milho


In [21]:
df.drop(['tipo'], axis = 1,inplace=True)

In [22]:
df.set_index('date',inplace=True)

In [23]:
df=df.sort_index(axis=0)
print(df.head(10))

       producao
date           
2017  125905263
2018  124131656
2019  121360171
2020  127453417
2021  130477261
2022  133791633


In [24]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,15
1,Target,producao
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(6, 1)"
5,Transformed data shape,"(6, 1)"
6,Transformed train set shape,"(5, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [25]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})


In [26]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})


In [34]:
top4 = exp.compare_models(n_select = 4)


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
croston,Croston,1.6052,1.5450,3833307.3923,3833307.3923,0.0303,0.0304,0.0200
naive,Naive Forecaster,1.6989,1.6459,3962858.3333,3962858.3333,0.0313,0.0317,0.2033
grand_means,Grand Means Forecaster,1.7654,1.6917,4359103.2500,4359103.2500,0.0343,0.0347,0.0600
theta,Theta Forecaster,1.8032,1.7308,4498025.7204,4498025.7204,0.0352,0.0361,0.0633
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,1.8851,1.7968,4882814.3000,4882814.3000,0.0381,0.0391,5.0567
xgboost_cds_dt,Extreme Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.8851,1.7968,4882814.3021,4882814.3021,0.0381,0.0391,0.5533
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,1.8851,1.7968,4882814.3000,4882814.3000,0.0381,0.0391,2.7167
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.8851,1.7968,4882816.5430,4882816.5430,0.0381,0.0391,0.3200
polytrend,Polynomial Trend Forecaster,1.8881,1.8038,4831249.0000,4831249.0000,0.0377,0.0387,0.0333
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,1.9096,1.8185,4969619.3900,4969619.3900,0.0388,0.0398,0.0767


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data.
 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

In [28]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":10})


####  xgboost_cds_dt Extreme Gradient Boosting w/ Cond. Deseasonalize & Detrending

In [29]:
xgboost = exp.create_model('xgboost_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = xgboost, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,0.5626,0.5626,997878.0000,997878.0000,0.0082,0.0082
1,2019.0000,3.4617,3.3812,7866853.0937,7866853.0937,0.0617,0.0637
2,2020.0000,1.6310,1.4466,5783711.8125,5783711.8125,0.0443,0.0453
Mean,nan,1.8851,1.7968,4882814.3021,4882814.3021,0.0381,0.0391
SD,nan,1.1971,1.1770,2875693.1863,2875693.1863,0.0223,0.0231


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


#### gbr_cds_dt   Gradient Boosting w/ Cond. Deseasonalize & Detrending

In [30]:
gbr_cds_dt = exp.create_model('gbr_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = gbr_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,15:11:34
Status,. . . . . . . . . . . . . . . . . .,Initializing CV
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting w/ Cond. Deseasonalize & Det...


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,0.5626,0.5626,997878.0000,997878.0000,0.0082,0.0082
1,2019.0000,3.4617,3.3812,7866859.6263,7866859.6263,0.0617,0.0637
2,2020.0000,1.6310,1.4466,5783712.0029,5783712.0029,0.0443,0.0453
Mean,nan,1.8851,1.7968,4882816.5430,4882816.5430,0.0381,0.0391
SD,nan,1.1971,1.1770,2875695.4658,2875695.4658,0.0223,0.0231


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


#### rf_cds_dt   Random Forest w/ Cond. Deseasonalize & Detrending

In [31]:
rf_cds_dt = exp.create_model('rf_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = rf_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,15:11:37
Status,. . . . . . . . . . . . . . . . . .,Selecting Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest w/ Cond. Deseasonalize & Detrending


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,0.5626,0.5626,997878.0000,997878.0000,0.0082,0.0082
1,2019.0000,3.5122,3.4305,7981608.9700,7981608.9700,0.0626,0.0646
2,2020.0000,1.7980,1.5948,6376049.2950,6376049.2950,0.0489,0.0501
Mean,nan,1.9576,1.8626,5118512.0883,5118512.0883,0.0399,0.0410
SD,nan,1.2094,1.1860,2986544.7633,2986544.7633,0.0231,0.0239


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.



## ada_cds_dt   AdaBoost w/ Cond. Deseasonalize & Detrendin


In [32]:
ada_cds_dt = exp.create_model('ada_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ada_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2018.0000,0.5626,0.5626,997878.0000,997878.0000,0.0082,0.0082
1,2019.0000,3.6812,3.5956,8365792.0000,8365792.0000,0.0656,0.0679
2,2020.0000,1.6310,1.4466,5783711.9000,5783711.9000,0.0443,0.0453
Mean,nan,1.9583,1.8683,5049127.3000,5049127.3000,0.0394,0.0405
SD,nan,1.2940,1.2736,3052458.0345,3052458.0345,0.0237,0.0246


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


### ALL GRAPHS

In [33]:
exp.plot_model(estimator = xgboost, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = gbr_cds_dt , plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = rf_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ada_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})